In [1]:
import requests
import pandas as pd
import io, base64
from PIL import Image

import os
import glob

import numpy as np              # for numerical computationss
import seaborn as sns
import torch                    # Pytorch module 
import matplotlib.pyplot as plt # for plotting informations on graph and images using tensors
import torch.nn as nn           # for creating  neural networks
from torch.utils.data import DataLoader # for dataloaders 
from PIL import Image           # for checking images
import torch.nn.functional as F # for functions for calculating loss
import torchvision.transforms as transforms   # for transforming images into tensors 
from torchvision.utils import make_grid       # for data checking
from torchvision.datasets import ImageFolder  # for working with classes and images
from torchsummary import summary              # for getting the summary of our model
import tensorflow as ts 
from  tensorflow import keras
import itertools
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

%matplotlib inline

C:\Users\david\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'sklearn'

### Take images with no result from webhost

In [3]:
url = 'https://agrodronesdelcampo.000webhostapp.com/ver_img.php?'
myobj = {'user': 'all'}
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
x = requests.request("POST",url, data = myobj, headers=headers)

data =x.json()

In [4]:
df = pd.DataFrame(data)

In [5]:
df

,ID,USERNAME,NAME_IMG,BASE_64,RESULT,CREATED_AT
0,14,ronaldo,hola_michael,/9j/4QBqRXhpZgAATU0AKgAAAAgABAEAAAQAAAABAAABAA...,Corn_(maize)___Common_rust_ 99.9%,2022-12-12 02:44:19
1,15,ronaldo,hoja_manzana,/9j/4QBqRXhpZgAATU0AKgAAAAgABAEAAAQAAAABAAABAA...,Apple___Apple_scab 97.8%,2022-12-12 02:44:19
2,16,ronaldo,hoja_de_manzana,/9j/4QBqRXhpZgAATU0AKgAAAAgABAEAAAQAAAABAAABAA...,Apple___Apple_scab 89.8%,2022-12-12 02:45:46


In [6]:
for index, row in df.iterrows():
    if row['RESULT'] == None:
        img = Image.open(io.BytesIO(base64.decodebytes(bytes(row['BASE_64'], "utf-8"))))
        img.save(row['ID']+'.jpg')

### Load the model 

In [7]:
from tensorflow.keras.models import load_model
model_3 = keras.models.load_model("C:\\Users\\david\\OneDrive\\Escritorio\\SP2\\PLANTS\\model_3.h5")
model_3.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_46 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 conv2d_47 (Conv2D)          (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 42, 42, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_48 (Conv2D)          (None, 42, 42, 64)        18496     
                                                                 
 conv2d_49 (Conv2D)          (None, 42, 42, 64)        36928     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                 

### Test the model with test data

In [8]:
Li = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 
      'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 
      'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 
      'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 
      'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 
      'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch',
      'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 
      'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
      'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']

# predicting an image
import os
import matplotlib.pyplot as plt
from keras.preprocessing import image
import numpy as np
directory="C:\\Users\\david\\OneDrive\\Escritorio\\SP2\\PLANTS\\API"
files = [os.path.join(directory,p) for p in sorted(os.listdir(directory))]
for i in range(0,40):
    image_path = files[i]
    print(image_path)
    start = image_path.find("API\\") + len("API\\")

    end = image_path.find(".jpg")

    id_img = image_path[start:end]

    print(id_img)
    new_img =keras.utils.load_img(image_path, target_size=(128, 128))
    img = keras.utils.img_to_array(new_img)
    img = np.expand_dims(img, axis=0)
    img = img/255
    prediction = model_3.predict(img)
    probabilty = prediction.flatten()
    max_prob = probabilty.max()*100
    index=prediction.argmax(axis=-1)[0]
    class_name = Li[index]
    #ploting image with predicted class name        
    plt.figure(figsize = (4,4))
    plt.imshow(new_img)
    plt.axis('off')
    plt.title(class_name+" "+ str(max_prob)[0:4]+"%")
    plt.show()
    
    url = 'https://agrodronesdelcampo.000webhostapp.com/trabajar_img.php?'
    myobj = {'id': id_img,
             'res':class_name+" "+ str(max_prob)[0:4]+"%"}
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    x = requests.request("POST",url, data = myobj, headers=headers)

IndexError: list index out of range

### DELETE ALL IMAGES

In [9]:
files = glob.glob('C:\\Users\\david\\OneDrive\\Escritorio\\SP2\\PLANTS\\API\\*')
for f in files:
    os.remove(f)